In [ ]:
from itertools import islice

import numpy as np

import holoviews as hv
from holoviews import opts
from holoviews.plotting.links import RangeToolLink

hv.extension('bokeh')

This demo demonstrates how to link two plots using the RangeToolLink along both axes. This can be useful to get an overview and a detailed view of some data at the same time.

## Declare data

In [ ]:
def mandelbrot_generator(h,w, maxit, bounds):
    """Generator that yields the mandlebrot set."""
    (l,b,r,t) = bounds
    y,x = np.ogrid[b:t : h*1j, l:r:w*1j]
    c = x+y*1j
    z = c
    divtime = maxit + np.zeros(z.shape, dtype=int)
    for i in range(maxit):
        z  = z**2 + c
        diverge = z*np.conj(z) > 2**2
        div_now = diverge & (divtime==maxit)
        divtime[div_now] = i
        z[diverge] = 2
        yield divtime

def mandelbrot(h,w, n, maxit, bounds):
    """Returns the mandelbrot set computed to maxit"""
    iterable =  mandelbrot_generator(h,w, maxit, bounds)
    return next(islice(iterable, n, None))

bounds = (-2,-1.4,0.8,1.4)
mbset = mandelbrot(800, 800, 45, 46, bounds)

mbset_image = hv.Image(mbset, bounds=bounds)

## Declare plot

Having declared an ``Image`` of the Mandelbrot set we make a smaller and larger version of it. The smaller ``source`` will serve as an overview containing the ``RangeTool`` which allows selecting the region to show in the larger ``target`` plot. We can control which axes should be linked to the ``RangeTool`` with the axes parameter on the ``RangeToolLink``:

In [ ]:
src = mbset_image.clone(link=False).opts(width=300, height=300)
tgt = mbset_image.opts(width=500, height=500, xaxis=None, yaxis=None, clone=True)
# Declare a RangeToolLink between the x- and y-axes of the two plots
RangeToolLink(src, tgt, axes=['x', 'y'])

(tgt + src).opts(
    opts.Image(default_tools=[]),
    opts.Layout(shared_axes=False))